#### Using Keras Tuner

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt

In [2]:
def build_model(hp):
    
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    
    model = Sequential([
        Dense(units, activation="relu"),
        Dense(10, activation="softmax")
    ])
    
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    return model

In [3]:
tuner = kt.BayesianOptimization(build_model, objective="val_accuracy", max_trials=5, 
                                executions_per_trial=2, directory="mnist_kt_test", overwrite=True)

In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [5]:
(x_train, y_train), (x_test, y_test) = load_data()

In [6]:
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255.0
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255.0

x_train_full = x_train[:]
y_train_full = y_train[:]

In [7]:
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

In [8]:
callbacks = [EarlyStopping(monitor="val_loss", patience=5)]

In [9]:
tuner.search(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_val, y_val), callbacks=callbacks, verbose=2)

Trial 5 Complete [00h 01m 04s]
val_accuracy: 0.9559500217437744

Best val_accuracy So Far: 0.9740000069141388
Total elapsed time: 00h 04m 35s
